In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

from utilities2014 import *

import time

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from scipy.spatial.distance import cdist, pdist, squareform
from scipy.cluster.hierarchy import average, fcluster, leaders, complete, single, dendrogram, ward

from joblib import Parallel, delayed

from skimage.color import gray2rgb
from skimage.util import img_as_float, pad
from skimage.morphology import disk
from skimage.filters.rank import gradient

from collections import defaultdict, Counter
from itertools import combinations, chain, product

import networkx
from networkx import from_dict_of_lists, dfs_postorder_nodes

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from skimage.feature import peak_local_max

def find_peaks(vs_max, topk=3):
    
    vs_max_smooth = gaussian_filter(vs_max, sigma=10)

    peaks = peak_local_max(vs_max_smooth)
    ypeaks = peaks[:,0]
    xpeaks = peaks[:,1]

    order = np.argsort(vs_max_smooth[ypeaks, xpeaks])[::-1]
    ypeaks = ypeaks[order]
    xpeaks = xpeaks[order]

    return [(x, y, vs_max[y, x]) for y, x in zip(ypeaks, xpeaks)[:topk]]

def find_peaks_all_rotations(vs_max_all_angles, topk=3):
    
    topk_locs = []
    for theta, vs_max in enumerate(vs_max_all_angles):
        topk_locs += map(lambda t: t+(theta,), find_peaks(vs_max, topk))
        
    topk_locs = sorted(topk_locs, key=lambda x: x[2], reverse=True)[:topk]
    
    return topk_locs

In [3]:
thetas = np.linspace(-np.pi/4, np.pi/4, 9)
n_theta = len(thetas)

In [4]:
dms = [DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=i) for i in range(7)]
for dm in dms:
    dm._load_image()

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [5]:
atlas_landmark_desc = 'RS141_0001_yuncong_08212015001655'

In [42]:
from enum import Enum

class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [44]:
with open('/home/yuncong/project/DavidData2014labelings/RS141/0001/RS141_0001_yuncong_08212015001655.pkl', 'r') as f:
    labeling = pickle.load(f)

In [9]:
def load_lm_vertices(lm_ind):
        
    vertices_all_rotations = [None for _ in range(n_theta)]
    for theta_i in range(n_theta):
        vertices_all_rotations[theta_i] = (striation_points_all_rotations[theta_i] if striation_points_all_rotations[theta_i] is not None else []) \
        + (textured_area_vertices_all_rotations[theta_i] if textured_area_vertices_all_rotations[theta_i] is not None else []) \
        + (boundary_vertices_all_rotations[theta_i] if boundary_vertices_all_rotations[theta_i] is not None else [])
            
    return (lm_ind, vertices_all_rotations)

landmark_indices = range(17)
landmark_vertices = dict(Parallel(n_jobs=16)(delayed(load_lm_vertices)(lm_ind) for lm_ind in landmark_indices))

In [ ]:
landmark_descriptors = dict(Parallel(n_jobs=16)(delayed(compute_landmark_descriptors_helper)(label, list(group)) 
                            for label, group in groupby(sorted(labelings['final_polygons'], key=itemgetter(0)),
                                                        itemgetter(0))))


all_polygon_vertices_global = [vertices for _, _, vertices in group]

all_vertices = np.vstack(all_polygon_vertices_global)

xmin, ymin = all_vertices.min(axis=0).astype(np.int)
xmax, ymax = all_vertices.max(axis=0).astype(np.int)
centroid_global = all_vertices.mean(axis=0).astype(np.int)

all_polygon_vertices_rotated_global_all_rotations = []

for theta_i in range(n_theta):
    all_polygon_vertices_rotated_global = [np.dot(vertices - centroid_global, Rs[theta_i].T).astype(np.int) + centroid_global
                                           for vertices in all_polygon_vertices_global]
    all_polygon_vertices_rotated_global_all_rotations.append(all_polygon_vertices_rotated_global)


In [27]:
def visualize_peaks(sec_ind, lm_ind, theta_i):

    response_map_all_rotations = dms[sec_ind].load_pipeline_result('responseMapLmAllRotations%d'%lm_ind, 'npy')

    lm_centroids_global = dms[sec_ind].load_pipeline_result('landmarkPositionsBeforeSnap', 'npy')
    cx, cy = lm_centroids_global[lm_ind]

    r = response_map_all_rotations[theta_i]
    
    top_locs = find_peaks(r, topk=-1)

    viz = img_as_ubyte(plt.cm.jet(r/r.max())[...,:3])

    for i, (x,y,s) in enumerate(top_locs):
        cv2.circle(viz, (x,y), 10, (0,0,0), -1)

    for i, (x,y,s) in enumerate(top_locs[:30]):
        cv2.putText(viz, '%d'%i, (x,y), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (0,0,0), 5)

    cv2.circle(viz, (cx,cy), 30, (255,255,255), -1)

    return viz

In [38]:
viz = visualize_peaks(sec_ind=5, lm_ind=1, theta_i=4)
display(viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [33]:
def visualize_top_responses(sec_ind, lm_ind):
    dm = dms[sec_ind]
    
    viz = img_as_ubyte(dm.image_rgb)
    
    response_map_all_rotations = dms[sec_ind].load_pipeline_result('responseMapLmAllRotations%d'%lm_ind, 'npy')
    top_locs = find_peaks_all_rotations(response_map_all_rotations, topk=10)
    
    with open('/home/yuncong/csd395/landmark_descriptors/%s_landmark_centroid_global_%d.pkl'%(atlas_landmark_desc, lm_ind), 'r') as f:
        lm_centroid_global = pickle.load(f)
    
    for i, (x, y, v, theta_i) in enumerate(top_locs):
                
        c = np.random.randint(0,255,3)
        
        for polygon_vs in landmark_vertices[lm_ind][int(theta_i)]:
            shifted_vertices = (polygon_vs + (x,y) - lm_centroid_global).astype(np.int, copy=False)
            for vx, vy in shifted_vertices:
                cv2.circle(viz, (vx, vy), 6, c, -1)
            cv2.polylines(viz, [shifted_vertices.astype(np.int)], True, c, 3)

        cv2.putText(viz, str(i), (x.astype(np.int), y.astype(np.int)), 
                    cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 3, c, 10)

    return viz

In [23]:
topPos_viz = visualize_top_responses(sec_ind=5, lm_ind=6)
display(topPos_viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [15]:
# def visualize_and_save(section_ind, lm_ind): 
#     viz = visualize_top_landmark_responses(section_ind, lm_ind)
#     dms[section_ind].save_pipeline_result(viz, 'topLocsLm%d'%lm_ind, 'jpg')

# landmark_indices = [0,5,6,7,8,9]
    
# _ = Parallel(n_jobs=16)(delayed(visualize_and_save)(section_ind, lm_ind) 
#                 for section_ind, lm_ind in product([0]+range(2,7), landmark_indices))

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm6.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm9.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm5.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm8.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm7.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0000/RS141_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_topLocsLm0.jpg
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014resu